In [66]:
import os

import dotenv
import pandas as pd
from tinkoff.invest import Client, GetAssetFundamentalsRequest
from tinkoff.invest.constants import INVEST_GRPC_API_SANDBOX
from tinkoff.invest.schemas import GetAssetReportsRequest

dotenv.load_dotenv('bank.env')
TOKEN = os.getenv('TOKEN')

In [67]:
money_to_invest = int(input("Enter the amount of money to invest: "))
qual_invest = int(input("Are you a qualified investor? Enter 1 for Yes, 0 for No: "))

with Client(TOKEN, target=INVEST_GRPC_API_SANDBOX) as client:
    shares = client.instruments.shares().instruments
    shares = [i for i in shares if i.currency == "rub" and (not i.for_qual_investor_flag or qual_invest)]

In [68]:
TARGET_COLUMNS = [
    "figi", "ticker", "lot", "name",
    "sector", "share_type", "liquidity_flag", "for_qual_investor_flag", "asset_uid", "price"
]
shares_df = pd.DataFrame(columns=TARGET_COLUMNS)
for i, share in enumerate(shares):
    shares_df.loc[i] = pd.Series(
        (
            share.figi,
            share.ticker,
            share.lot,
            share.name,
            share.sector,
            share.share_type,
            share.liquidity_flag,
            share.for_qual_investor_flag,
            share.asset_uid,
            None
        ), index=TARGET_COLUMNS
    )

In [69]:
from tinkoff.invest.schemas import TargetItem

with Client(TOKEN) as client:
    for i, share in enumerate(shares):
        try:
            share_forecasts = TargetItem(uid=share.uid)
            print(share_forecasts.recommendation.RECOMMENDATION_UNSPECIFIED)
        except:
            fundamentals_data = None

request = GetAssetReportsRequest()

In [70]:
fundamentals_columns = [
    "market_capitalization", "high_price_last_52_weeks", "low_price_last_52_weeks",
    "average_daily_volume_last_10_days", "average_daily_volume_last_4_weeks",
    "beta", "free_float", "forward_annual_dividend_yield", "revenue_ttm",
    "ebitda_ttm", "net_income_ttm", "eps_ttm", "diluted_eps_ttm",
    "free_cash_flow_ttm", "five_year_annual_revenue_growth_rate",
    "three_year_annual_revenue_growth_rate", "pe_ratio_ttm", "price_to_sales_ttm",
    "price_to_book_ttm", "price_to_free_cash_flow_ttm", "roe", "roa"
]

TARGET_COLUMNS.extend(fundamentals_columns)
shares_df = pd.DataFrame(columns=TARGET_COLUMNS)

with Client(TOKEN) as client:
    for i, share in enumerate(shares):
        shares_df.loc[i] = pd.Series(
            (
                share.figi,
                share.ticker,
                share.lot,
                share.name,
                share.sector,
                share.share_type,
                share.liquidity_flag,
                share.for_qual_investor_flag,
                share.asset_uid,
                None
            ), index=TARGET_COLUMNS[:10]
        )
        try:
            request = GetAssetFundamentalsRequest(assets=[share.asset_uid])
            fundamentals_data = client.instruments.get_asset_fundamentals(request=request).fundamentals[0]
        except:
            print(i, share.name, "UID not found")
            fundamentals_data = None

        for col in fundamentals_columns:
            shares_df.at[i, col] = getattr(fundamentals_data, col, None)

    for i, row in shares_df.iterrows():
        last_price_data = client.market_data.get_last_prices(figi=[row.figi]).last_prices[0]
        price = last_price_data.price.units + last_price_data.price.nano / 10e9
        shares_df.at[i, "price"] = price * row.lot

14eca4fd4f325250ac653eb062ea9fb6 GetAssetFundamentals NOT_FOUND 50009


41 Озон Фармацевтика UID not found


In [71]:
display(shares_df)

,figi,ticker,lot,name,sector,share_type,liquidity_flag,for_qual_investor_flag,asset_uid,price,...,diluted_eps_ttm,free_cash_flow_ttm,five_year_annual_revenue_growth_rate,three_year_annual_revenue_growth_rate,pe_ratio_ttm,price_to_sales_ttm,price_to_book_ttm,price_to_free_cash_flow_ttm,roe,roa
0,TCS00A108ZR8,DATA,1,Группа Аренадата,it,1,True,False,4cda1996-5fa5-4c56-9442-958e2eb20afc,118.036,...,0.0,0.0,0.0,0.0,0.0,0.0,24.69,0.0,0.0,0.0
1,BBG000FWGSZ5,IRKT,100,Яковлев,industrials,1,True,False,6faba6f9-1ba7-4c3b-92ad-a0ec9ddf1303,2904.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.99,0.0,-21.41,-2.76
2,BBG004S68CV8,VSMO,1,ВСМПО-АВИСМА,materials,1,True,False,f712e646-7c5d-4266-aabd-a18c9c65a7e2,23780.0,...,0.0,-2986306000.0,0.0,10.91,11.15,2.19,0.88,-92.97,8.29,4.82
3,BBG000Q7ZZY2,UNAC,1000,Объединенная авиастроительная корпорация,industrials,1,True,False,68af184b-07c1-4f27-8889-62d869f9dc0e,69.95,...,0.0,21392000000.0,0.0,3.33,0.0,1.41,11.55,33.28,-68.84,-3.07
4,TCS00A106YF0,VKCO,1,ВК,it,1,True,False,f0a6fe09-3981-4501-a8f5-91821935e4d4,300.04,...,0.0,0.0,0.0,0.0,0.0,0.0,0.62,0.0,-22.44,-9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,BBG004S68BR5,NMTP,100,НМТП,industrials,1,True,False,84cd22cb-6bbf-42cc-944c-e57af772d3d5,803.55,...,0.0,0.0,0.0,13.96,4.73,2.27,1.05,0.0,23.58,17.17
152,BBG000QFH687,TGKA,100000,ТГК-1,utilities,1,True,False,8558bad9-44a4-4f91-bca7-9c6e8bcd454f,68.24,...,0.0,0.0,0.0,3.42,0.0,0.0,0.17,0.0,4.96,3.79
153,BBG0027F0Y27,CNTLP,100,Центральный Телеграф - акции привилегированные,telecom,2,False,False,2dd88e4e-0531-41e5-8c41-14625ad84240,708.2,...,0.0,146024000.0,0.0,-7.13,63.96,0.34,0.54,2.97,0.82,0.57
154,BBG004S68FR6,MTLRP,10,Мечел - Привилегированные акции,materials,2,True,False,cb1c065d-9d85-4aeb-9e31-41fafb7db8e1,1040.2,...,0.0,28711000000.0,0.0,15.2,1.25,0.04,-0.2,0.51,-16.11,4.36


In [72]:
def analyze_and_select_portfolio(budget):
    def calculate_scores(row):

        if row["market_capitalization"] is None:
            return 0

        valuation_score = (
                                  (1 / row['pe_ratio_ttm'] if row['pe_ratio_ttm'] > 0 else 0) * 0.4 +
                                  (1 / row['price_to_sales_ttm'] if row['price_to_sales_ttm'] > 0 else 0) * 0.3 +
                                  (1 / row['price_to_book_ttm'] if row['price_to_book_ttm'] > 0 else 0) * 0.3
                          ) * 0.3

        growth_score = (
                               row['three_year_annual_revenue_growth_rate'] * 0.5 +
                               row['five_year_annual_revenue_growth_rate'] * 0.3 +
                               row['eps_ttm'] * 0.2
                       ) * 0.2

        profitability_score = (
                                      row['roe'] * 0.5 +
                                      row['roa'] * 0.3 +
                                      (row['net_income_ttm'] / row['market_capitalization'] if row[
                                                                                                   'market_capitalization'] > 0 else 0) * 0.2
                              ) * 0.2

        liquidity_score = (
                                  row['market_capitalization'] * 0.4 +
                                  row['average_daily_volume_last_10_days'] * 0.4 +
                                  row['free_float'] * 0.2
                          ) * 0.15

        risk_score = (
                             (1 / row['beta'] if row['beta'] > 0 else 0) * 0.5 +
                             ((row['high_price_last_52_weeks'] - row['low_price_last_52_weeks']) / row[
                                 'high_price_last_52_weeks'] if row['high_price_last_52_weeks'] > 0 else 0) * 0.5
                     ) * 0.15

        return round((valuation_score + growth_score + profitability_score + liquidity_score + risk_score) / 1e9, 3)

    shares_df["investment_score"] = shares_df.apply(calculate_scores, axis=1)

    def knapsack_optimization(shares, budget):
        shares.sort(key=lambda x: x["investment_score"] / x["price"], reverse=True)
        portfolio = []
        remaining_budget = budget

        for share in shares:
            price = share["price"]
            if price <= remaining_budget:
                portfolio.append(share)
                remaining_budget -= price

        return portfolio


    shares_to_consider = shares_df[["figi", "ticker", "investment_score", "price"]].to_dict("records")

    optimal_portfolio = knapsack_optimization(shares_to_consider, budget)

    return pd.DataFrame(optimal_portfolio)

In [73]:
s = 0
portfolio = analyze_and_select_portfolio(money_to_invest)

for i, row in portfolio.iterrows():
    s += row.iloc[3]
print(f"Money to spend on portfolio {round(s, 6)}, money remains: {round(money_to_invest - s, 6)}")
display(portfolio)

Money to spend on portfolio 99964.072, money remains: 35.928


,figi,ticker,investment_score,price
0,BBG004731354,ROSN,300.109,463.050
1,BBG000Q7ZZY2,UNAC,42.729,69.950
2,BBG004730ZJ9,VTBR,25.439,78.033
3,BBG004S684M6,SIBN,158.853,551.055
4,TCS00A102093,ELMT,4.513,16.060
...,...,...,...,...
91,BBG000LNHHJ9,KMAZ,4.922,1150.200
92,BBG004S68696,RASP,10.737,2670.850
93,TCS00A103X66,POSI,8.103,2027.000
94,BBG005D1WCQ1,QIWI,0.548,200.060


In [74]:
def save_to_csv(dataframe, filename="shares_dataset.csv"):
    dataframe.to_csv(filename, index=False)
    print(f"Dataset saved to {filename}")


save_to_csv(portfolio)


Dataset saved to shares_dataset.csv
